In [9]:
import json
from requests_html import AsyncHTMLSession
from utils import getStringBetweenTwoWords
from utils import extract_and_stringify_object , printJsonToFile
from Product import ProductDetailDTO
#rewrite scrape_Jarir_full function with the object extraction 
async def scrape_Jarir_full(url):  
    # session = AsyncHTMLSession()
    # response = await session.get(url)
    # soup=response.content.decode()
    # stringObj = getStringBetweenTwoWords(soup, 'original:', ',related').replace('!1', 'true').replace('!0', 'false')
    # obj = json.loads(extract_and_stringify_object(stringObj))
    #load the json from the file jarir.json
    obj = json.load(open("jarir.json"))
    printJsonToFile(obj, "jarir.json")
    name_global = obj.get("name", "")
    isAvailable = obj.get("check_availability_status", 0) == 1
    PriceAfterDiscount = obj.get("final_price", obj.get("price", None))
    description = obj.get("description", obj.get("short_description", ""))
    rating = None  
    images = ["https://ak-asset.jarir.com/akeneo-prod/asset/"+image["image"] for image in obj.get("media_gallery", []) if image.get("type") == "image"]
    item_data = ProductDetailDTO(name_Global=name_global, price=PriceAfterDiscount,images=images,productlink1=url,description_Global=description,is_available=isAvailable,rating=rating)
    return item_data
#Test
url = 'https://www.jarir.com/sa-en/default-category/msi-clae-a1m-gaming-consoles-and-handheld-631416.html'
x=await scrape_Jarir_full(url)
#print it when Finish
print(x)



ProductDetailDTO(prod_id=0, domainId=1, productlink1=https://www.jarir.com/sa-en/default-category/msi-clae-a1m-gaming-consoles-and-handheld-631416.html, status=, lastUpdated=2024-06-21 20:08:51.903693, lastScraped=2024-06-21 20:08:51.903693, id=0, name_Local=, name_Global=MSI Claw A1M Handheld Gaming Console, Intel Core Ultra 7 Processor, 512 GB, Black, description_Local=, description_Global=None, price=3799, rating=None, is_available=True, brand=, images=['https://ak-asset.jarir.com/akeneo-prod/asset/b/b/c/c/bbcc8c1c12a995f33ba9a9b8ab7c365703114777_631416.jpg', 'https://ak-asset.jarir.com/akeneo-prod/asset/f/4/f/e/f4fedbefb6cb954f43d7f6125bdbe259d7cf4d07_631416_1.jpg', 'https://ak-asset.jarir.com/akeneo-prod/asset/1/f/6/0/1f60183c0fc8f48082251e64b74fcc4dc7c74378_631416_2.jpg', 'https://ak-asset.jarir.com/akeneo-prod/asset/7/2/c/b/72cb5a45081b0696c661de124c2e0bb4758f2128_631416_3.jpg', 'https://ak-asset.jarir.com/akeneo-prod/asset/5/3/8/6/5386b16272b56e1c79cb085f7afc2167dacc6308_631416

In [2]:
# from requests_html import AsyncHTMLSession
# from utils import getStringBetweenTwoWords
# from Product import ProductDetailDTO

# async def scrape_Jarir_full(url):  
#     session = AsyncHTMLSession()
#     response = await session.get(url)
#     script_target_object=getStringBetweenTwoWords(response.content.decode(), 'window.__INITIAL_STATE__= ','</script>')
#     strat_index=response.content.decode().index('window.__INITIAL_STATE__')
#     end_index=response.content.decode().index('</script>',strat_index)
#     script_target_object=response.content.decode()[strat_index:end_index].replace('window.__INITIAL_STATE__=','')
#     price=getStringBetweenTwoWords(script_target_object, 'final_price_ex_tax:',',')
#     Title=getStringBetweenTwoWords(script_target_object, 'GTM_name:',',')
#     images_string = getStringBetweenTwoWords(script_target_object, 'media_gallery','tsk').split("},{")
#     images=[]
#     for i in range(len(images_string)):
#         images.append("https://ak-asset.jarir.com/akeneo-prod/asset/"+getStringBetweenTwoWords(images_string[i], 'image:"','",lab'))
#     item_data = ProductDetailDTO(name_Global=Title, price=price,images=images,productlink1=url)
#     return item_data

# #Test
# url = 'https://www.jarir.com/sa-en/default-category/msi-clae-a1m-gaming-consoles-and-handheld-631416.html'
# x=await scrape_Jarir_full(url)
# print(x.__dict__)
